In [1]:
import pandas as pd
# Read the CSV file
Data = pd.read_csv(".\Dataset-Mental-Disorders-Clean.csv")
features = list(Data.columns)
print(f"features: {features}")

patient_number = Data.iloc[:,0]
Dataset = Data.iloc[:,1:-1]
print(f"x: {list(Dataset.columns)}")
y = Data.iloc[:,-1]
print(f"y: {y.name}")

import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

features: ['Patient Number', 'Sadness', 'Euphoric', 'Exhausted', 'Sleep dissorder', 'Mood Swing', 'Suicidal thoughts', 'Anorxia', 'Authority Respect', 'Try-Explanation', 'Aggressive Response', 'Ignore & Move-On', 'Nervous Break-down', 'Admit Mistakes', 'Overthinking', 'Sexual Activity', 'Concentration', 'Optimisim', 'Expert Diagnose']
x: ['Sadness', 'Euphoric', 'Exhausted', 'Sleep dissorder', 'Mood Swing', 'Suicidal thoughts', 'Anorxia', 'Authority Respect', 'Try-Explanation', 'Aggressive Response', 'Ignore & Move-On', 'Nervous Break-down', 'Admit Mistakes', 'Overthinking', 'Sexual Activity', 'Concentration', 'Optimisim']
y: Expert Diagnose


In [2]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

X_train, X_test, y_train, y_test = train_test_split(Dataset, y, test_size=0.33, random_state=1)

Creation of a simple Neural Network Multi class classifier

In [3]:
from sklearn.preprocessing import OneHotEncoder
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)  # `sparse=False` returns dense array

# Perform one-hot encoding
y_test_hot = encoder.fit_transform(y_test.to_numpy())
y_train_hot = encoder.fit_transform(y_train.to_numpy())

ValueError: Expected 2D array, got 1D array instead:
array=[1 1 3 1 1 3 2 1 0 2 3 1 0 3 1 0 3 3 0 0 0 0 3 1 3 0 2 2 1 3 0 3 2 3 3 0 1
 0 3 3].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
## NN
import tensorflow as tf
from tensorflow import keras
from keras import Model
# from keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Conv2D
# from tensorflow.keras.layers import Reshape
# from tensorflow.keras.layers import Conv2DTranspose
# from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.callbacks import LossCallback

# # object for callback function during training
# loss_callback = LossCallback()

l1 = 64
activation = 'relu'
alpha = 0.05
epochs = 15
# define model
print(f"Dataset.shape[1]: {Dataset.shape[1]}")
print(f"y.shape[1]: {y.shape[1]}")
inputs = Input(shape=(Dataset.shape[1],))
hidden1 = Dense(l1, activation=activation)(inputs)
hidden2 = Dense(l1, activation=activation)(hidden1)
predictions = Dense(y_train_hot.shape[1], activation="softmax")(hidden2)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

## compile model with mse loss and ADAM optimizer (uncomment for SGD)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=alpha), metrics=['accuracy'])
history = model.fit(x=X_train, y=y_train_hot, epochs=epochs, verbose=0, batch_size=32)#, callbacks=[loss_callback])

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

NN_pred = model.predict(y_test_hot)
# Create the confusion matrix
nn_cm = confusion_matrix(y_test_hot, NN_pred)
accuracy = accuracy_score(y_test_hot, NN_pred)
precision = precision_score(y_test_hot, NN_pred, average='macro')
recall = recall_score(y_test_hot, NN_pred, average='macro')
f1 = f1_score(y_test_hot, NN_pred, average='macro')

print(nn_cm)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)